In [75]:
from specutils import Spectrum1D
from astropy.nddata import StdDevUncertainty
from astropy.io import fits
from datetime import date
from pandas import to_datetime
import pandas as pd
#from scripts.ingests.utils import * #scripts do not need anything from ingest
import numpy.ma as ma
import matplotlib.pyplot as plt
import os
import astropy.units as u

In [76]:
from astroquery.vizier import Vizier
from astroquery.simbad import Simbad
from astroquery.xmatch import XMatch
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from astropy.coordinates import SkyCoord # a "sky coordinate" object. Helpful for dealing with locations in space or in the sky

# Astropy's Table tools
from astropy.table import Table, vstack, join

In [111]:
from csv import writer
from csv import reader
import csv 

In [133]:
vizier_code = "J/A+A/564/A55/table1" # store the vizier table code into a variable

query_catalog = Vizier() # Create a vizier query object

query_catalog.ROW_LIMIT = -1 # Turn off the row limit

manja2014 = query_catalog.get_catalogs(vizier_code)[0] #downloads the table

manja2014
RA = list(manja2014['RAJ2000'][1:7])
dec = list(manja2014['DEJ2000'][1:7])

In [124]:
df = pd.read_csv("Manja14_spectra - Sheet1.csv")
df['RA'] = RA
df['dec'] = dec
list(df.columns)
df.to_csv('Manja14_spectra_adjusted.csv', index = False)

In [138]:
from astropy.table import Table #need to import astropy.table to get Table to work
#table_file = 'Manja14_spectra - Sheet1.csv'
table_file='Manja14_spectra_adjusted.csv'
data = Table.read(table_file)

In [139]:
import astropy
import seaborn as sns
from astropy.io import fits
from astropy.io import ascii
from glob import glob
import astropy.io.fits as pyfits

original_data_dir = 'ISAAC_Manjavacas2014/' #got rid of temp_data at begining of directory
fits_data_dir = 'Manj14/'

    
for row in data:
    object = row['Source'] #row is defined so it loops through each row to 
    #logger.debug(f'source: {object}')
    
    spectrum_url = row['Spectrum'] #goes to specific column entry finds the url 

    file = os.path.basename(spectrum_url)
    
    full_name = (original_data_dir + file)
    
    files = glob('ISAAC_Manjavacas2014/*.fits')

    for file_name in files:
        data = pyfits.getdata(file_name)
        name = file_name[:-len('.fits')] # Remove .fits from the file name
    
    spectrum_table = Table(data)
    obs_date = to_datetime(row['observation_date'])
    
    instrument = row['instrument'] 
    telescope = row['telescope']
  #  author = 'Manjavacas et al 2019, AJ, 157, 101'         looking for this info
    reference_doi = '10.1051/0004-6361/201323016'     
    comment = row['comments'] #changed from spectrum comments
    title = 'ISAAC Spectra of Brown Dwarfs from Manjavacus 2014'
    bandpass = 'JHK' #get from paper
    bibcode = '2014A&A...564A..55M'
    RA = row['RA'] #from the vizier catalog, put in csv table
    dec = row['dec'] #from the vizier catalog, put in csv table
    
    history1 = f'Original file: {file}' #gives orginal .csv name of file
    history2 = 'This file generated by SIMPLE-db/scripts/ingests/rewrite_spectra.py' #shows where file came from 
    
    wavelength_unit = u.micron  #using astropy to defien units 
    flux_unit = u.erg/u.cm/u.cm/u.s/u.Angstrom
    
    #only 1 columns, does not have flux uncertaitnty 
    wavelength = spectrum_table['col0'] * wavelength_unit   #multiplying everythign by untis to convert
    flux = spectrum_table['col1'] * flux_unit
    #TOOK OUT FLUX_UNC BC MANJA 14 ONLY HAD 2 COLUMNS
    
    spectrum_data_out = Table({'wavelength': wavelength, 'flux':flux }) #TOOK OUT FLUX_UNC
    
    hdu1 = fits.BinTableHDU(data = spectrum_data_out) 
    hdu1.header['EXTNAME'] = 'SPECTRUM'

    # Build the header
    header = fits.Header()
    header.set('EXTNAME', 'PRIMARY', 'name of this extension')
    # IVOA SpectrumDM keywords REQUIRED
    header.set('VOCLASS', 'Spectrum-1.0','VO Data Model') ##########################
    header.set('VOPUB', 'NOT SURE WHAT THIS SHOULD BE','VO Publisher ID URI') ##########################
    header.set('TITLE', title, 'data set title') 
    header.set('OBJECT', object, 'name of observed object')
    header.set('RA', RA , '[deg] Pointing position') 
    header.set('DEC', dec , '[deg] Pointing position') 
    header.set('TMID', 'time', '[d] MJD mid expsoure') ##########################
    header.set('TELAPSE', 'exposure time','Total elapsed time') ##########################
    header.set('SPEC_VAL', bandpass, '[angstrom] Characteristic spec coord')
    header.set('SPEC_BW', (max(wavelength).value - min(wavelength).value), '[angstrom] Width of spectrum')
    header.set('TDMIN1', min(wavelength).value) #NEED TO ADD DESCRIPTION
    header.set('TDMAX1', max(wavelength).value) #NEED TO ADD DESCRIPTION
    # Other IVOA SpectrumDM keywords
    header.set('VOREF', bibcode, 'bibcode' )
    header.set('DATE', date.today().strftime("%Y-%m-%d"), 'date of file creation')
    header.set('INSTRUME', instrument, 'name of instrument')
    header.set('DATE-OBS', obs_date.strftime("%Y-%m-%d"), 'date of the observation')
   #header.set('AUTHOR', author, 'author of the data') ##########################
    header.set('REFERENC', reference_doi, 'bibliographic reference')
    header.set('TELESCOP', telescope, 'name of telescope')
    if not ma.is_masked(comment): ####################################################
        header.set('COMMENT', comment)
    header.set('HISTORY', history1) #NEED TO ADD DESCRIPTION but this is not in the pdf
    header.set('HISTORY', history2) #NEED TO ADD DESCRIPTION but this is not in the pdf
    
    hdu0 = fits.PrimaryHDU(header=header) #not sure what this menas 

    spectrum_mef = fits.HDUList([hdu0,hdu1])
    
    file_root = os.path.splitext(file)[0] #split the path name into a pair root and ext so the root is just the ext [0] is the name of the file wihtout the .csv
    fits_filename = fits_data_dir + file_root + '.fits' #turns into fits files by putting it in new folder that we defined at begining and adding name of file then .fits
    try:
        spectrum_mef.writeto(fits_filename, overwrite=True)
        # SHOULD BE: spectrum.write(fits_filename, format='tabular-fits', overwrite=True, update_header=True)
        #logger.info(f'Wrote {fits_filename}')
    except:
        raise SimpleError
        
        # Read the spectrum back in and check
spec1d_fits_rt = fits.open(fits_filename)
header_rt = spec1d_fits_rt[0].header
spec1d_rt = Spectrum1D.read(fits_filename) #reads the fits file